In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
!pip install tensorboardX
!git clone https://github.com/GuyKabiri/language_models


fatal: destination path 'language_models' already exists and is not an empty directory.


In [ ]:
!pip3 install scipy==1.2.1
import scipy
print(scipy.__version__)

     |████████████████████████████████| 24.8MB 188kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


1.4.1


In [ ]:
import keras

import keras.backend as K
from keras.datasets import imdb
from keras.layers import  LSTM, Embedding, TimeDistributed, Input, Dense
from keras.models import Model
from tensorflow.python.client import device_lib

from tqdm import tqdm
import os, random
import functools as fc

from argparse import ArgumentParser

import numpy as np

from tensorboardX import SummaryWriter

import language_models.util as util

CHECK = 5

In [ ]:
def generate_seq(model : Model, seed, size, temperature=1.0):
    """
    :param model: The complete RNN language model
    :param seed: The first few wordas of the sequence to start generating from
    :param size: The total size of the sequence to generate
    :param temperature: This controls how much we follow the probabilities provided by the network. For t=1.0 we just
        sample directly according to the probabilities. Lower temperatures make the high-probability words more likely
        (providing more likely, but slightly boring sentences) and higher temperatures make the lower probabilities more
        likely (resulting is weirder sentences). For temperature=0.0, the generation is _greedy_, i.e. the word with the
        highest probability is always chosen.
    :return: A list of integers representing a samples sentence
    """

    ls = seed.shape[0]

    # Due to the way Keras RNNs work, we feed the model a complete sequence each time. At first it's just the seed,
    # zero-padded to the right length. With each iteration we sample and set the next character.

    tokens = np.concatenate([seed, np.zeros(size - ls)])

    for i in range(ls, size):

        probs = model.predict(tokens[None,:])

        # Extract the i-th probability vector and sample an index from it
        next_token = util.sample_logits(probs[0, i-1, :], temperature=temperature)

        tokens[i] = next_token

    return [int(t) for t in tokens]


In [ ]:
class Args1:
  epochs = 20 # Number of epochs
  embedding_size = 300 # Size of the word embeddings on the input layer.
  out_every = 1 # Output every n epochs.
  lr = 0.001 # Learning rate
  batch = 128 # Batch size
  task = 'wikisimple'
  data = './data' # Data file. Should contain one sentence per line.
  lstm_capacity = 256
  max_length = None # Sentence max length.
  top_words = 10000 # Word list size.
  limit = None # Character cap for the corpus - not relevant in our exercise.
  tb_dir = './runs/words' # Tensorboard directory
  seed = -1 # RNG seed. Negative for random (seed is printed for reproducability).
  extra = 1 # Number of extra LSTM layers.
  go_backward=False #go in the oppsite direction

class Args2:
  epochs = 20 # Number of epochs
  embedding_size = 300 # Size of the word embeddings on the input layer.
  out_every = 1 # Output every n epochs.
  lr = 0.001 # Learning rate
  batch = 128 # Batch size
  task = 'wikisimple'
  data = './data' # Data file. Should contain one sentence per line.
  lstm_capacity = 256
  max_length = None # Sentence max length.
  top_words = 10000 # Word list size.
  limit = None # Character cap for the corpus - not relevant in our exercise.
  tb_dir = './runs/words' # Tensorboard directory
  seed = -1 # RNG seed. Negative for random (seed is printed for reproducability).
  extra = 1 # Number of extra LSTM layers.
  go_backward=True #go in the oppsite direction


class Args3:
  epochs = 20 # Number of epochs
  embedding_size = 300 # Size of the word embeddings on the input layer.
  out_every = 1 # Output every n epochs.
  lr = 0.001 # Learning rate
  batch = 128 # Batch size
  task = 'wikisimple'
  data = './data' # Data file. Should contain one sentence per line.
  lstm_capacity = 256
  max_length = None # Sentence max length.
  top_words = 10000 # Word list size.
  limit = None # Character cap for the corpus - not relevant in our exercise.
  tb_dir = './runs/words' # Tensorboard directory
  seed = -1 # RNG seed. Negative for random (seed is printed for reproducability).
  extra = 2 # Number of extra LSTM layers.
  go_backward=False #go in the oppsite direction

class Args4:
  epochs = 20 # Number of epochs
  embedding_size = 300 # Size of the word embeddings on the input layer.
  out_every = 1 # Output every n epochs.
  lr = 0.001 # Learning rate
  batch = 128 # Batch size
  task = 'wikisimple'
  data = './data' # Data file. Should contain one sentence per line.
  lstm_capacity = 256
  max_length = None # Sentence max length.
  top_words = 10000 # Word list size.
  limit = None # Character cap for the corpus - not relevant in our exercise.
  tb_dir = './runs/words' # Tensorboard directory
  seed = -1 # RNG seed. Negative for random (seed is printed for reproducability).
  extra = 2 # Number of extra LSTM layers.
  go_backward=True #go in the oppsite direction


options1 = Args1()
options2 = Args2()
options3 = Args3()
options4 = Args4()

In [ ]:
def perplexity(y_true, y_pred):
    """
    The perplexity metric. Why isn't this part of Keras yet?!
    https://stackoverflow.com/questions/41881308/how-to-calculate-perplexity-of-rnn-in-tensorflow
    https://github.com/keras-team/keras/issues/8267
    """
    cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred)
    perplexity = K.exp(cross_entropy)
    return perplexity


In [ ]:
def sparse_loss(y_true, y_pred):
    return K.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def go(options):

    tbw = SummaryWriter(log_dir=options.tb_dir)

    if options.seed < 0:
        seed = random.randint(0, 1000000)
        print('random seed: ', seed)
        np.random.seed(seed)
    else:
        np.random.seed(options.seed)

    if options.task == 'wikisimple':

        x, w2i, i2w = \
            util.load_words(util.DIR + '/datasets/wikisimple.txt', vocab_size=options.top_words, limit=options.limit)
        x_train=x[0:len(x)*8//10]
        x_validate=x[(len(x)*8//10)+1:len(x)*9//10]
        x_test=x[(len(x)*9//10)+1:-1]
        # Finding the length of the longest sequence
        x_max_len = max([len(sentence) for sentence in x])
        
        numwords = len(i2w)
        print('max sequence length ', x_max_len)
        print(numwords, 'distinct words')


        x = util.batch_pad(x, options.batch, add_eos=True)
        x_train = util.batch_pad(x_train, options.batch, add_eos=True)
        x_validate = util.batch_pad(x_validate, options.batch, add_eos=True)
        x_test = util.batch_pad(x_test, options.batch, add_eos=True)
    elif options.task == 'file':

        x, w2i, i2w = \
            util.load_words(options.data_dir, vocab_size=options.top_words, limit=options.limit)

        # Finding the length of the longest sequence
        x_max_len = max([len(sentence) for sentence in x])

        numwords = len(i2w)
        print('max sequence length ', x_max_len)
        print(numwords, 'distinct words')

        x = util.batch_pad(x, options.batch, add_eos=True)

    else:
        raise Exception('Task {} not recognized.'.format(options.task))

    def decode(seq):
        return ' '.join(i2w[id] for id in seq)

    print('Finished data loading. ', sum([b.shape[0] for b in x]), ' sentences loaded')

    ## Define model

    input = Input(shape=(None, ))
    embedding = Embedding(numwords, options.lstm_capacity, input_length=None)

    embedded = embedding(input)

    decoder_lstm = LSTM(options.lstm_capacity, return_sequences=True,go_backwards=options.go_backward)
    h = decoder_lstm(embedded)

    if options.extra is not None:
        for _ in range(options.extra):
            h = LSTM(options.lstm_capacity, return_sequences=True,go_backwards=options.go_backward)(h)

    fromhidden = Dense(numwords, activation='linear')
    out = TimeDistributed(fromhidden)(h)

    model = Model(input, out)

    opt = keras.optimizers.Adam(lr=options.lr)
    lss = sparse_loss

    model.compile(opt, lss)
    model.summary()

    ## Training

    #- Since we have a variable batch size, we make our own training loop, and train with
    #  model.train_on_batch(...). It's a little more verbose, but it gives us more control.

    epoch = 0
    instances_seen = 0
    while epoch < options.epochs:

        for batch in tqdm(x_train):
            n, l = batch.shape

            batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)  # prepend start symbol
            batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)     # append pad symbol

            loss = model.train_on_batch(batch_shifted, batch_out[:, :, None])

            instances_seen += n
            tbw.add_scalar('lm/batch-loss', float(loss), instances_seen)

        epoch += 1

        # Show samples for some sentences from random batches
        for temp in [0.0, 0.9, 1, 1.1, 1.2]:
            print('### TEMP ', temp)
            for i in range(CHECK):
                b = random.choice(x)

                if b.shape[1] > 20:
                    seed = b[0,:20]
                else:
                    seed = b[0, :]

                seed = np.insert(seed, 0, 1)
                gen = generate_seq(model, seed,  60, temperature=temp)

                print('*** [', decode(seed), '] ', decode(gen[len(seed):]))
    preplexity=0
    count=0
    for batch in tqdm(x_validate):
      n, l = batch.shape

      batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)  # prepend start symbol
      batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)     # append pad symbol

      valid_pred = model.predict_on_batch(batch_shifted)
      preplexity+=tf.math.reduce_mean(perplexity(batch_out,valid_pred))
      count+=1
    print("x_validate preplexity: {}".format(preplexity/count))
    preplexity=0
    count=0
    for batch in tqdm(x_train):
      n, l = batch.shape

      batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)  # prepend start symbol
      batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)     # append pad symbol

      valid_pred = model.predict_on_batch(batch_shifted)
      preplexity+=tf.math.reduce_mean(perplexity(batch_out,valid_pred))
      count+=1
    print("x_train preplexity: {}".format(preplexity/count))
    preplexity=0
    count=0
    for batch in tqdm(x_test):
      n, l = batch.shape

      batch_shifted = np.concatenate([np.ones((n, 1)), batch], axis=1)  # prepend start symbol
      batch_out = np.concatenate([batch, np.zeros((n, 1))], axis=1)     # append pad symbol

      valid_pred = model.predict_on_batch(batch_shifted)
      preplexity+=tf.math.reduce_mean(perplexity(batch_out,valid_pred))
      count+=1
    print("x_test preplexity: {}".format(preplexity/count))
    return model,i2w,w2i


In [ ]:
def next_word(trained_model,i2w):
  chosen_word=input("please write a word")
  tokens = np.concatenate([seed, np.zeros(1)])
  probs = trained_model.predict(tokens[None,:])
  # Extract the 1st probability vector and sample an index from it
  next_token = util.sample_logits(probs[0, 1, :], temperature=1.0)
  return i2w[next_token]


def gen_seq(model : Model, seed,ls, size, i2w,temperature=1.0):
    """
    :param model: The complete RNN language model
    :param seed: The first few wordas of the sequence to start generating from
    :param size: The total size of the sequence to generate
    :param temperature: This controls how much we follow the probabilities provided by the network. For t=1.0 we just
        sample directly according to the probabilities. Lower temperatures make the high-probability words more likely
        (providing more likely, but slightly boring sentences) and higher temperatures make the lower probabilities more
        likely (resulting is weirder sentences). For temperature=0.0, the generation is _greedy_, i.e. the word with the
        highest probability is always chosen.
    :return: A list of integers representing a samples sentence
    """

    # Due to the way Keras RNNs work, we feed the model a complete sequence each time. At first it's just the seed,
    # zero-padded to the right length. With each iteration we sample and set the next character.

    tokens = np.concatenate([seed, np.zeros(size - ls)])

    for i in range(ls, size):

        probs = model.predict(tokens[None,:])

        # Extract the i-th probability vector and sample an index from it
        next_token = util.sample_logits(probs[0, i-1, :], temperature=temperature)

        tokens[i] = next_token
    return ' '.join(i2w[int(id)] for id in tokens)
    

def seq_probality(trained_model,sequence,w2i):
  tokens=[w2i[word] for word in sequence.split()]
  probs=trained_model.predict(tokens)
  mean_prob=0
  for i in probs:
    mean_prob+=i
  mean_prob=mean_prob[0]
  return mean_prob[len(mean_prob)-1]/100

In [19]:
print('OPTIONS', options1)
model1,i2w_1,w2i_1=go(options1)

OPTIONS <__main__.Args1 object at 0x7fd6ed6b8048>
random seed:  842076
raw data read
max sequence length  132
10000 distinct words
max length per batch:  [15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 3

  0%|          | 0/186 [00:00<?, ?it/s]

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, None, 256)         525312    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
time_distributed (TimeDistri (None, None, 10000)       2570000   
Total params: 6,180,624
Trainable params: 6,180,624
Non-trainable params: 0
_________________________________________________________________


100%|██████████| 186/186 [05:48<00:00,  1.87s/it]


### TEMP  0.0
*** [ <START> the district was made in december 1993 by joining the old districts of <UNK> <UNK> and <UNK> but also including ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
*** [ <START> a schismatic is a person who creates schism in an organization or who is a member of a <UNK> group ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
*** [ <START> francis ii of france lrb january 19 <UNK> december 5 <UNK> rrb was a king of france and a member ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UN

...

x_validate preplexity: 762277632.0


  0%|          | 0/24 [00:00<?, ?it/s]

x_train preplexity: 717992128.0


100%|██████████| 24/24 [00:25<00:00,  1.06s/it]

x_test preplexity: 777240640.0


In [20]:
print('OPTIONS', options2)
model2,i2w_2,w2i_2=go(options2)

OPTIONS <__main__.Args2 object at 0x7fd6ed6b8080>
random seed:  241513
raw data read
max sequence length  132
10000 distinct words
max length per batch:  [15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 3

  0%|          | 0/186 [00:00<?, ?it/s]

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 256)         2560000   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 10000)       2570000   
Total params: 6,180,624
Trainable params: 6,180,624
Non-trainable params: 0
_________________________________________________________________


100%|██████████| 186/186 [05:50<00:00,  1.89s/it]


### TEMP  0.0
*** [ <START> a student of camille saint sa ëns he served as the organist of saint <UNK> church in paris for 62 ]  permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation
*** [ <START> the night before he priest died he said in a soft voice o death o holy death come do come ]  permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutation permutat

...

x_validate preplexity: 242687888.0


  0%|          | 0/24 [00:00<?, ?it/s]

x_train preplexity: 5763720.0


100%|██████████| 24/24 [00:22<00:00,  1.07it/s]

x_test preplexity: 254432304.0


In [21]:
print('OPTIONS', options3)
model3,i2w_3,w2i_3=go(options3)

OPTIONS <__main__.Args3 object at 0x7fd73901f5c0>
random seed:  972152
raw data read
max sequence length  132
10000 distinct words
max length per batch:  [15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 3

  0%|          | 0/186 [00:00<?, ?it/s]

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 256)         2560000   
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 10000)       2570000   
Total params: 6,705,936
Trainable params: 6,705,936
Non-trainable params: 0
_________________________________________________

100%|██████████| 186/186 [06:42<00:00,  2.16s/it]


### TEMP  0.0
*** [ <START> <UNK> involving sound from the <UNK> are not found in most dialects of english though they are found in many ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
*** [ <START> in the presidents case the disease caused <UNK> and <UNK> <UNK> the <UNK> com <EOS> <PAD> ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
*** [ <START> a popular writer in her time her work has received even more attention from literary critics in recent years <EOS> ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UN

...

x_validate preplexity: 975690048.0


  0%|          | 0/24 [00:00<?, ?it/s]

x_train preplexity: 1037709952.0


100%|██████████| 24/24 [00:25<00:00,  1.07s/it]

x_test preplexity: 1006653504.0


In [22]:
print('OPTIONS', options4)
model4,i2w_4,w2i_4=go(options4)

OPTIONS <__main__.Args4 object at 0x7fd73901f588>
random seed:  558588
raw data read
max sequence length  132
10000 distinct words
max length per batch:  [15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 3

  0%|          | 0/186 [00:00<?, ?it/s]

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 256)         2560000   
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
lstm_9 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 10000)       2570000   
Total params: 6,705,936
Trainable params: 6,705,936
Non-trainable params: 0
_________________________________________________

100%|██████████| 186/186 [06:51<00:00,  2.21s/it]


### TEMP  0.0
*** [ <START> it is found in the region nord pas de calais in the pas de calais department in the north of ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
*** [ <START> in the presidents case the disease caused <UNK> and <UNK> <UNK> the <UNK> com <EOS> <PAD> ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
*** [ <START> its discovery was announced by scott s sheppard david c jewitt jan kleyna and brian g marsden on may 4 ]  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <

...

x_validate preplexity: 1174910848.0


  0%|          | 0/24 [00:00<?, ?it/s]

x_train preplexity: 1206404608.0


100%|██████████| 24/24 [00:24<00:00,  1.03s/it]

x_test preplexity: 1222468736.0


In [24]:
seed=np.asarray([w2i_3['i'],w2i_3['love']])
seq1=gen_seq(model3,seed,2,7,i2w_3,temperature=0.1)
seq2=gen_seq(model3,seed,2,7,i2w_3,temperature=1.0)
seq3=gen_seq(model3,seed,2,7,i2w_3,temperature=10.0)
print("seq1",seq1)
print("seq2",seq2)
print("seq3",seq3)

seq1_prob=abs(seq_probality(model3,seq1,w2i_3))
seq2_prob=abs(seq_probality(model3,seq2,w2i_3))
seq3_prob=abs(seq_probality(model3,seq3,w2i_3))
print("sequence 1 probability is ",seq1_prob)
print("sequence 2 probability is ",seq2_prob)
print("sequence 3 probability is ",seq3_prob)

seq1 i love 1973 nme ghali burgess oddparents
seq2 i love symphonic subdivision attitude hartman neighbor
seq3 i love fans feels midlands angola lachlan
sequence 1 probability is  0.04357423782348633
sequence 2 probability is  0.05840563297271729
sequence 3 probability is  0.05222022533416748
